In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime
import os
import requests
import json
import logging
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from data_upload_utils import upload_to_github, update_airtable, create_airtable_record, delete_file_from_github

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Initialize CoinGecko with retry mechanism
session = requests.Session()
retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))
cg = CoinGeckoAPI(session=session)

# Validate environment variables
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
GITHUB_TOKEN = os.getenv("GH_TOKEN")
if not all([AIRTABLE_API_KEY, GITHUB_TOKEN]):
    logger.error("Missing environment variables: AIRTABLE_API_KEY or GITHUB_TOKEN")
    raise ValueError("Required environment variables are not set")

# Define coins
coins = {
    "BTC": "bitcoin",
    "ETH": "ethereum",
    "ADA": "cardano",
    "SOL": "solana",
    "DOT": "polkadot",
    "AVAX": "avalanche-2",
}

# Airtable configuration
BASE_ID = "appnssPRD9yeYJJe5"
TABLE_NAME = "Database"
AIRTABLE_URL = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"
AIRTABLE_HEADERS = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json",
}

# GitHub configuration
GITHUB_REPO = "SagarFieldElevate/DatabaseManagement"
BRANCH = "main"
UPLOAD_PATH = "Uploads"

def fetch_coingecko_data():
    """Fetch 365 days of price data and compute metrics."""
    data = []
    for symbol, coin_id in coins.items():
        try:
            market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency="usd", days=365)
            prices = market_data.get("prices", [])
            if not prices:
                logger.warning(f"No price data for {symbol}")
                continue

            for i in range(1, len(prices)):
                prev_day = prices[i - 1]
                current_day = prices[i]

                prev_timestamp, prev_price = prev_day
                current_timestamp, current_price = current_day

                # Ensure prices are valid
                if not (prev_price > 0 and current_price > 0):
                    logger.warning(f"Invalid price data for {symbol} at {current_timestamp}")
                    continue

                high = max(prev_price, current_price)
                low = min(prev_price, current_price)
                volatility = ((high - low) / low) * 100
                trading_range = high - low

                data.append({
                    "symbol": symbol,
                    "timestamp": datetime.utcfromtimestamp(current_timestamp / 1000).isoformat(),
                    "high_24h_usd": round(high, 2),
                    "low_24h_usd": round(low, 2),
                    "volatility_24h_%": round(volatility, 2),
                    "trading_range_24h_usd": round(trading_range, 2),
                })
        except Exception as e:
            logger.error(f"Failed to fetch data for {symbol}: {str(e)}")
            continue
    return data

def get_airtable_record():
    """Fetch Airtable record with filtering."""
    try:
        params = {"filterByFormula": "{Name}='365-Day Volatility and Range'"}
        response = requests.get(AIRTABLE_URL, headers=AIRTABLE_HEADERS, params=params)
        response.raise_for_status()
        records = response.json().get("records", [])
        if len(records) > 1:
            logger.warning("Multiple records found with Name='365-Day Volatility and Range'. Using first.")
        return records[0] if records else None
    except Exception as e:
        logger.error(f"Failed to fetch Airtable record: {str(e)}")
        raise

def ensure_data_format(data):
    """Ensure data entries have consistent format."""
    formatted_data = []
    for entry in data:
        try:
            formatted_entry = entry.copy()
            if isinstance(formatted_entry.get("timestamp"), datetime):
                formatted_entry["timestamp"] = formatted_entry["timestamp"].isoformat()
            for key in ["high_24h_usd", "low_24h_usd", "volatility_24h_%", "trading_range_24h_usd"]:
                if key in formatted_entry:
                    formatted_entry[key] = round(float(formatted_entry[key]), 2)
            formatted_data.append(formatted_entry)
        except Exception as e:
            logger.warning(f"Skipping malformed data entry: {str(e)}")
    return formatted_data

def merge_unique_data(existing_data, new_data):
    """Merge new data with existing, keeping only unique entries."""
    existing_set = {(d["symbol"], d["timestamp"]) for d in existing_data}
    unique_new = [d for d in new_data if (d["symbol"], d["timestamp"]) not in existing_set]
    logger.info(f"Found {len(unique_new)} unique new entries out of {len(new_data)}")
    return existing_data + unique_new

def update_or_create_airtable(record_id, data, raw_url, filename):
    """Update or create Airtable record."""
    payload = {
        "fields": {
            "Name": "365-Day Volatility and Range",
            "Data": data,
            "Database Attachment": [{"url": raw_url, "filename": filename}],
        }
    }
    try:
        if record_id:
            logger.info(f"Updating existing Airtable record: {record_id}")
            update_airtable(record_id, raw_url, filename, AIRTABLE_URL, AIRTABLE_API_KEY)
            # Update Data field separately to ensure all fields are updated
            patch_url = f"{AIRTABLE_URL}/{record_id}"
            patch_payload = {"fields": {"Data": data, "Name": "365-Day Volatility and Range"}}
            response = requests.patch(patch_url, headers=AIRTABLE_HEADERS, json=patch_payload)
        else:
            logger.info("Creating new Airtable record")
            create_airtable_record(raw_url, filename, AIRTABLE_URL, AIRTABLE_API_KEY)
            # Update Data field for new record
            response = requests.get(AIRTABLE_URL, headers=AIRTABLE_HEADERS, params={
                "filterByFormula": "{Name}='365-Day Volatility and Range'"
            })
            response.raise_for_status()
            record_id = response.json()["records"][0]["id"]
            patch_url = f"{AIRTABLE_URL}/{record_id}"
            patch_payload = {"fields": {"Data": data}}
            response = requests.patch(patch_url, headers=AIRTABLE_HEADERS, json=patch_payload)
        response.raise_for_status()
        logger.info("Successfully updated/created Airtable record")
    except Exception as e:
        logger.error(f"Airtable operation failed: {str(e)}")
        raise

def main():
    filename = None
    try:
        # Fetch data
        new_data = fetch_coingecko_data()
        if not new_data:
            logger.error("No data fetched from CoinGecko")
            return

        # Create DataFrame and Excel file
        df = pd.DataFrame(new_data)
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        filename = f"historical_volatility_trading_range_365_days_{timestamp}.xlsx"
        df.to_excel(filename, index=False)
        logger.info(f"Created Excel file: {filename}")

        # Get existing Airtable record
        record = get_airtable_record()
        record_id = record["id"] if record else None
        existing_data = ensure_data_format(record["fields"].get("Data", []) if record else [])

        # Merge unique data
        merged_data = merge_unique_data(existing_data, new_data)

        # Upload to GitHub
        github_response = upload_to_github(filename, GITHUB_REPO, BRANCH, UPLOAD_PATH, GITHUB_TOKEN)
        raw_url = github_response["content"]["download_url"]
        file_sha = github_response["content"]["sha"]
        logger.info("Uploaded file to GitHub")

        # Update or create Airtable record
        update_or_create_airtable(record_id, merged_data, raw_url, filename)

        # Cleanup
        delete_file_from_github(filename, GITHUB_REPO, BRANCH, UPLOAD_PATH, GITHUB_TOKEN, file_sha)
        os.remove(filename)
        logger.info("Cleaned up local and GitHub files")
        filename = None

    except Exception as e:
        logger.error(f"Script failed: {str(e)}")
        if filename and os.path.exists(filename):
            os.remove(filename)
            logger.info("Removed local file due to error")
        raise

if __name__ == "__main__":
    main()